# Imports

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
cd /content/drive/My\ Drive/Colab\ Notebooks/ML_A

/content/drive/My Drive/Colab Notebooks/ML_A


In [115]:
# !pip install catboost

In [116]:
from datetime import datetime
import numpy as np
import pandas as pd
import math
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from statistics import mean

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn import model_selection


RANDOM_SEED = 0

# Read Dataset

In [117]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [118]:
 np.count_nonzero(test_df.isnull())  

0


# Fill or Remove missing values

In [119]:
# Drop rows with null
train_df = train_df.dropna(how = 'any')
train_df.dropna(how = 'any', inplace=True)

#fill rows with null by mean conditioned by label
# x = train_df.fillna(train_df.groupby("label").transform("mean"))

x = train_df
x_test = test_df

In [120]:
x

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
5,189129552,10.5,3407.0,182.0,0.0000,112.0,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,correct


In [121]:
x['label'] = x['label'].map({'correct' :1, 'incorrect':0})

In [122]:
x.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
count,1.696800e+04,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000
mean,2.014695e+08,13.721437,1702.836987,628.875000,32.045122,112.489510,6.965160,79.998018,6.968278,79.997108,312.657884,0.910066
std,7.453627e+06,98.992493,8121.556167,8064.657846,465.564240,442.129943,0.210840,0.242832,0.380253,0.360503,606.372103,0.286096
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,45.307700,0.000000,0.000000
25%,1.947342e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.864830,79.870475,6.865580,79.870100,125.500000,1.000000
50%,2.013928e+08,10.500000,1009.000000,123.000000,0.115600,73.000000,6.913970,79.898600,6.915940,79.899000,211.050000,1.000000
75%,2.083314e+08,10.500000,1802.250000,351.000000,14.334400,144.000000,7.042698,79.956500,7.046555,79.960000,366.205000,1.000000
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,48.132000,81.009700,26945.990000,1.000000


In [123]:
x_test.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare
count,8.576000e+03,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000,8576.000000
mean,2.180067e+08,11.088095,1541.590485,508.502099,24.671655,119.863106,6.969246,80.008728,6.970008,80.010145,293.278057
std,2.789252e+06,5.359602,5952.458574,5784.005855,333.515641,793.233190,0.217809,0.251873,0.218826,0.252229,436.730282
min,2.132846e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.943130,79.817700,5.936600,79.818000,0.000000
25%,2.156247e+08,10.500000,540.000000,30.000000,0.000000,26.000000,6.864810,79.871600,6.864950,79.871300,126.077500
50%,2.179006e+08,10.500000,982.500000,116.000000,0.000000,72.000000,6.913605,79.900500,6.915460,79.902250,203.950000
75%,2.203904e+08,10.500000,1723.000000,336.250000,11.744200,138.000000,7.068875,79.963225,7.073505,79.967825,344.075000
max,2.228607e+08,160.500000,447230.000000,446934.000000,25760.361800,55571.000000,8.425010,80.776400,8.421730,80.805500,26844.330000


In [124]:
x.corr()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,1.000000,0.019134,-0.019753,-0.016872,-0.017291,-0.017930,0.028994,0.029901,0.006327,0.030312,-0.014647,0.069251
additional_fare,0.019134,1.000000,-0.000774,0.197606,0.199606,0.002352,0.009120,0.000612,0.004642,0.001532,0.315285,-0.041040
duration,-0.019753,-0.000774,1.000000,0.969788,0.968667,0.040577,-0.027081,-0.025590,-0.015969,-0.013603,0.865256,-0.113149
meter_waiting,-0.016872,0.197606,0.969788,1.000000,0.999474,0.030450,-0.016167,-0.014196,-0.010114,-0.007895,0.870126,-0.107379
meter_waiting_fare,-0.017291,0.199606,0.968667,0.999474,1.000000,0.029911,-0.015009,-0.012467,-0.009426,-0.006794,0.865073,-0.103418
meter_waiting_till_pickup,-0.017930,0.002352,0.040577,0.030450,0.029911,1.000000,0.011381,0.016483,0.006942,0.011047,0.045468,-0.035546
pick_lat,0.028994,0.009120,-0.027081,-0.016167,-0.015009,0.011381,1.000000,0.519962,0.558312,0.336847,-0.024726,-0.027702
pick_lon,0.029901,0.000612,-0.025590,-0.014196,-0.012467,0.016483,0.519962,1.000000,0.304976,0.651794,-0.035472,-0.038129
drop_lat,0.006327,0.004642,-0.015969,-0.010114,-0.009426,0.006942,0.558312,0.304976,1.000000,-0.420574,-0.015615,-0.011633
drop_lon,0.030312,0.001532,-0.013603,-0.007895,-0.006794,0.011047,0.336847,0.651794,-0.420574,1.000000,-0.014981,-0.030311


# Date times to separate features

In [125]:
def dates(date, need):
    if date == None or date == '':
        return date
    else:
        d_t = date.split(" ")
        d = d_t[0]
        t = d_t[1]

        d_l = d.split("/")
        day = int(d_l[1])
        month = int(d_l[0])
        year = int(d_l[2])

        t_l = t.split(':')
        hour = int(t_l[0])
        mini = int(t_l[1])
        if need == 'y':
            return year
        if need == 'mo':
            return month
        if need == 'd':
            return day
        if need == 'h':
            return hour
        if need =='min': 
            return mini

def pick_drop_date_diff(pick, drop):
  pda = pick.split(" ")[0]
  dda = drop.split(" ")[0]

  if pda == dda:
    return 0
  else:
    return 1


# Date Diff

In [126]:
# x['pickup_day'] = x['pickup_time'].apply(dates, need = "d")
# x['pickup_hour'] = x['pickup_time'].apply(dates, need = "h")
# x['pickup_mini'] = x['pickup_time'].apply(dates, need = "min")

# x_test['pickup_day'] = x_test['pickup_time'].apply(dates, need = "d")
# x_test['pickup_hour'] = x_test['pickup_time'].apply(dates, need = "h")
# x_test['pickup_mini'] = x_test['pickup_time'].apply(dates, need = "min")

# x['drop_day'] = x['drop_time'].apply(dates, need = "d")
# x['drop_hour'] = x['drop_time'].apply(dates, need = "h") 
# x['drop_mini'] = x['drop_time'].apply(dates, need = "min")

# x_test['drop_day'] = x_test['drop_time'].apply(dates, need = "d")
# x_test['drop_hour'] = x_test['drop_time'].apply(dates, need = "h")
# x_test['drop_mini'] = x_test['drop_time'].apply(dates, need = "min")

x['pick_drop_date_diff'] = x.apply(lambda x: pick_drop_date_diff(x.pickup_time, x.drop_time), axis=1)
x_test['pick_drop_date_diff'] = x_test.apply(lambda x: pick_drop_date_diff(x.pickup_time, x.drop_time), axis=1)


# ~~Create cyclic features from year,month,date, hour,minute~~

In [127]:
# PICKUP TIME 
# x['pickup_year'] = x['pickup_time'].apply(dates, need = "y")

# x['pickup_month'] = x['pickup_time'].apply(dates, need = "mo")
# x['pickup_day'] = x['pickup_time'].apply(dates, need = "d")
# x['pickup_hour'] = x['pickup_time'].apply(dates, need = "h")
# x['pickup_mini'] = x['pickup_time'].apply(dates, need = "min")

# x['pickup_month_sin'] = np.sin((x.pickup_month-1)*(2.*np.pi/12))
# x['pickup_month_cos'] = np.cos((x.pickup_month-1)*(2.*np.pi/12))

# x['pickup_day_sin'] = np.sin((x.pickup_day-1)*(2.*np.pi/31))
# x['pickup_day_cos'] = np.cos((x.pickup_day-1)*(2.*np.pi/31))

# x['pickup_hour_sin'] = np.sin(x.pickup_hour*(2.*np.pi/24))
# x['pickup_hour_cos'] = np.cos(x.pickup_hour*(2.*np.pi/24))

# x['pickup_mini_sin'] = np.sin(x.pickup_mini*(2.*np.pi/60))
# x['pickup_mini_cos'] = np.cos(x.pickup_mini*(2.*np.pi/60))

# x = x.drop('pickup_month', axis=1)
# x = x.drop('pickup_day', axis=1)
# x = x.drop('pickup_hour', axis=1)
# x = x.drop('pickup_mini', axis=1)

# x_test['pickup_year'] = x_test['pickup_time'].apply(dates, need = "y")

# x_test['pickup_month'] = x_test['pickup_time'].apply(dates, need = "mo")
# x_test['pickup_day'] = x_test['pickup_time'].apply(dates, need = "d")
# x_test['pickup_hour'] = x_test['pickup_time'].apply(dates, need = "h")
# x_test['pickup_mini'] = x_test['pickup_time'].apply(dates, need = "min")

# x_test['pickup_month_sin'] = np.sin((x_test.pickup_month-1)*(2.*np.pi/12))
# x_test['pickup_month_cos'] = np.cos((x_test.pickup_month-1)*(2.*np.pi/12))

# x_test['pickup_day_sin'] = np.sin((x_test.pickup_day-1)*(2.*np.pi/31))
# x_test['pickup_day_cos'] = np.cos((x_test.pickup_day-1)*(2.*np.pi/31))

# x_test['pickup_hour_sin'] = np.sin(x_test.pickup_hour*(2.*np.pi/24))
# x_test['pickup_hour_cos'] = np.cos(x_test.pickup_hour*(2.*np.pi/24))

# x_test['pickup_mini_sin'] = np.sin(x_test.pickup_mini*(2.*np.pi/60))
# x_test['pickup_mini_cos'] = np.cos(x_test.pickup_mini*(2.*np.pi/60))

# x_test = x_test.drop('pickup_month', axis=1)
# x_test = x_test.drop('pickup_day', axis=1)
# x_test = x_test.drop('pickup_hour', axis=1)
# x_test = x_test.drop('pickup_mini', axis=1)

# # DROP TIME

# x['drop_year'] = x['drop_time'].apply(dates, need = "y")

# x['drop_month'] = x['drop_time'].apply(dates, need = "mo")
# x['drop_day'] = x['drop_time'].apply(dates, need = "d")
# x['drop_hour'] = x['drop_time'].apply(dates, need = "h")
# x['drop_mini'] = x['drop_time'].apply(dates, need = "min")

# x['drop_month_sin'] = np.sin((x.drop_month-1)*(2.*np.pi/12))
# x['drop_month_cos'] = np.cos((x.drop_month-1)*(2.*np.pi/12))

# x['drop_day_sin'] = np.sin((x.drop_day-1)*(2.*np.pi/31))
# x['drop_day_cos'] = np.cos((x.drop_day-1)*(2.*np.pi/31))

# x['drop_hour_sin'] = np.sin(x.drop_hour*(2.*np.pi/24))
# x['drop_hour_cos'] = np.cos(x.drop_hour*(2.*np.pi/24))

# x['drop_mini_sin'] = np.sin(x.drop_mini*(2.*np.pi/60))
# x['drop_mini_cos'] = np.cos(x.drop_mini*(2.*np.pi/60))

# x = x.drop('drop_month', axis=1)
# x = x.drop('drop_day', axis=1)
# x = x.drop('drop_hour', axis=1)
# x = x.drop('drop_mini', axis=1)

# x_test['drop_year'] = x_test['drop_time'].apply(dates, need = "y")

# x_test['drop_month'] = x_test['drop_time'].apply(dates, need = "mo")
# x_test['drop_day'] = x_test['drop_time'].apply(dates, need = "d")
# x_test['drop_hour'] = x_test['drop_time'].apply(dates, need = "h")
# x_test['drop_mini'] = x_test['drop_time'].apply(dates, need = "min")

# x_test['drop_month_sin'] = np.sin((x_test.drop_month-1)*(2.*np.pi/12))
# x_test['drop_month_cos'] = np.cos((x_test.drop_month-1)*(2.*np.pi/12))

# x_test['drop_day_sin'] = np.sin((x_test.drop_day-1)*(2.*np.pi/31))
# x_test['drop_day_cos'] = np.cos((x_test.drop_day-1)*(2.*np.pi/31))

# x_test['drop_hour_sin'] = np.sin(x_test.drop_hour*(2.*np.pi/24))
# x_test['drop_hour_cos'] = np.cos(x_test.drop_hour*(2.*np.pi/24))

# x_test['drop_mini_sin'] = np.sin(x_test.drop_mini*(2.*np.pi/60))
# x_test['drop_mini_cos'] = np.cos(x_test.drop_mini*(2.*np.pi/60))

# x_test = x_test.drop('drop_month', axis=1)
# x_test = x_test.drop('drop_day', axis=1)
# x_test = x_test.drop('drop_hour', axis=1)
# x_test = x_test.drop('drop_mini', axis=1)

# ~~Line distanece~~

In [128]:
# x['distance'] = (x['drop_lat']- x['pick_lat'])**2 + (x['drop_lon'] - x['pick_lon'])**2
# x_test['distance'] = (x_test['drop_lat']- x_test['pick_lat'])**2 + (x_test['drop_lon'] - x_test['pick_lon'])**2

# Get distance

In [129]:
def get_real_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    lon1 = np.radians(lon1)
    lon2 = np.radians(lon2)

    lat_diff = lat2 - lat1
    lon_diff = lon2 - lon1

    a = np.sin(lat_diff/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(lon_diff/2)**2
    c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    return R*c

x['distance'] = get_real_distance(x['pick_lat'],x['pick_lon'],x['drop_lat'],x['drop_lon'])
x_test['distance'] = get_real_distance(x_test['pick_lat'],x_test['pick_lon'],x_test['drop_lat'],x_test['drop_lon'])

#~~ Mobile time feature~~

In [130]:
# x['mobile_time'] = x['duration'] - x['meter_waiting']
# x_test['mobile_time'] = x_test['duration'] - x_test['meter_waiting']

# ~~Fare for mobile time feature~~

In [131]:
# x['fare_4_mobile_time'] = x['fare'] - x['meter_waiting_fare'] - x['additional_fare']
# x_test['fare_4_mobile_time'] = x_test['fare'] - x_test['meter_waiting_fare'] -  x_test['additional_fare']

# ~~ Average speed ~~

In [132]:
# def higher_average_speed(dis, dur):
#   if dur==0:
#     spd = 10000
#   else:
#     spd = dis/dur
#   if spd > 500 and spd <1:
#     return 0
#   else:
#     return 1


In [133]:
# x['higher_speed'] = x.apply(lambda x: higher_average_speed(x.distance, x.duration), axis=1)
# x_test['higher_speed'] = x_test.apply(lambda x: higher_average_speed(x.distance, x.duration), axis=1)

# Drop columns

In [134]:
trip_id_no_norm = x_test['tripid']
# 'tripid'
# 'additional_fare'
# duration
# meter_waiting
# meter_waiting_fare
# meter_waiting_till_pickup
# pickup_time
# drop_time
# pick_lat
# pick_lon
# drop_lat
# drop_lon
# fare

# x = x.drop('label', axis=1)

x = x.drop('pickup_time', axis=1)
x = x.drop('drop_time', axis=1)
# x = x.drop('additional_fare', axis=1)
# x = x.drop('duration', axis=1)
# x = x.drop('meter_waiting', axis=1)
# x = x.drop('meter_waiting_fare', axis=1)
# x = x.drop('meter_waiting_till_pickup', axis=1)
x = x.drop('tripid', axis=1)
x = x.drop(['pick_lat','drop_lat','pick_lon','drop_lon'], axis=1) #'pickup_time', 'drop_time',
# x = x.drop('meter_waiting_till_pickup', axis=1)

x_test = x_test.drop('pickup_time', axis=1)
x_test = x_test.drop('drop_time', axis=1)
# x_test = x_test.drop('additional_fare', axis=1)
# x_test = x_test.drop('duration', axis=1)
# x_test = x_test.drop('meter_waiting', axis=1)
# x_test = x_test.drop('meter_waiting_fare', axis=1)
# x_test = x_test.drop('meter_waiting_till_pickup', axis=1)
x_test = x_test.drop('tripid', axis=1)
x_test = x_test.drop(['pick_lat','drop_lat','pick_lon','drop_lon'], axis=1) #'pickup_time', 'drop_time',
# x_test = x_test.drop('meter_waiting_till_pickup', axis=1)

In [135]:
x.shape

(16968, 9)

# Normalize data

In [136]:
# x_mean = x.mean()
# x_std = x.std()
# x_norm = (x - x_mean)/x_std
# x = x_norm

# for i in x.columns[1:]:
#     i_mean = x[i].mean()
#     i_std = x[i].std()
#     i_norm = (x[i] - i_mean)/i_std
#     x[i] = i_norm

# i_mean = x['pickup_time_new'].mean()
# i_std = x['pickup_time_new'].std()
# i_norm = (x['pickup_time_new'] - i_mean)/i_std
# x['pickup_time_new'] = i_norm

# i_mean = x_test['pickup_time_new'].mean()
# i_std = x_test['pickup_time_new'].std()
# i_norm = (x_test['pickup_time_new'] - i_mean)/i_std
# x_test['pickup_time_new'] = i_norm

# x_test_mean = x_test.mean()
# x_test_std = x_test.std()
# x_test_norm = (x_test - x_test_mean)/x_test_std
# x_test = x_test_norm

# for i in x_test.columns[1:]:
#     i_mean = x_test[i].mean()
#     i_std = x_test[i].std()
#     i_norm = (x_test[i] - i_mean)/i_std
#     x_test[i] = i_norm

# i_mean = x['drop_time_new'].mean()
# i_std = x['drop_time_new'].std()
# i_norm = (x['drop_time_new'] - i_mean)/i_std
# x['drop_time_new'] = i_norm

# i_mean = x_test['drop_time_new'].mean()
# i_std = x_test['drop_time_new'].std()
# i_norm = (x_test['drop_time_new'] - i_mean)/i_std
# x_test['drop_time_new'] = i_norm

#standard scaler

In [137]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(x)

# x = scaler.transform(x)
# x_test = scaler.transform(x_test)

In [138]:
# import seaborn as sns
# sns.boxplot(x=x['distance'])

In [139]:
# x.max()

In [140]:
# x_test.max()

In [141]:
# x.describe()

In [142]:
# x_test.describe()

In [143]:
# np.sum(x['meter_waiting_fare'] > x['fare'])

In [144]:
# x['mobile_time'] = x['duration'] - x['meter_waiting']
# x_test['mobile_time'] = x_test['duration'] - x_test['meter_waiting']

# Remove outliers




In [145]:
# from scipy import stats
# # x_ = x[(np.abs(stats.zscore(x.loc[:, x.columns != 'label'])) < 3).all(axis=1)]
# x_test_ = x_test[(np.abs(stats.zscore(x_test)) < 3).all(axis=1)]

# x_ = x[(np.abs(stats.zscore(x['additional_fare'])) < 3)]
# x_test_ = x_test[(np.abs(stats.zscore(x_test['additional_fare'])) < 3)]

In [146]:
# x_.max()

In [147]:
# x_test_.max()

# Label Outliers

In [148]:
# from scipy import stats
# x['fare_ok'] = np.where(np.abs(stats.zscore(x['fare']))<3,1,0)
# x_test['fare_ok'] = np.where(np.abs(stats.zscore(x_test['fare']))<3,1,0)

# x['additional_fare_ok'] = np.where(np.abs(stats.zscore(x['additional_fare']))<5,1,0)
# x_test['additional_fare_ok'] = np.where(np.abs(stats.zscore(x_test['additional_fare']))<5,1,0)

# x.loc[not np.abs(stats.zscore(x['fare']))>3, 'fare'] = 0
# x.loc[np.abs(stats.zscore(x['meter_waiting']))>3, 'meter_waiting'] = x['meter_waiting'].mean()
# x.loc[np.abs(stats.zscore(x['meter_waiting_fare']))>3, 'meter_waiting_fare'] = x['meter_waiting_fare'].mean()
# x.loc[np.abs(stats.zscore(x['meter_waiting_till_pickup']))>3, 'meter_waiting_till_pickup'] = x['meter_waiting_till_pickup'].mean()
# x.loc[np.abs(stats.zscore(x['fare']))>3, 'fare'] = x['fare'].mean()
# x.loc[np.abs(stats.zscore(x['distance']))>3, 'distance'] = x['distance'].mean()

# x['additional_fare'].clip_upper(x_['additional_fare'].max())
# x_test['additional_fare'].clip_upper(x_test_['additional_fare'].max())

# x.loc[(np.abs(stats.zscore(x.loc[:, x.columns != 'label']))>3), x] = x_.max()
# x_test.loc[(np.abs(stats.zscore(x_test))>3), x_test] = x_test_.max()


In [149]:
# x.fare_ok.value_counts()
# x.additional_fare_ok.value_counts()


In [150]:
# x_test.fare_ok.value_counts()
# x_test.additional_fare_ok.value_counts()

# y

In [151]:
y = x['label']

# Preprocessed DataFrame

In [152]:
x

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,pick_drop_date_diff,distance
0,10.5,834.0,56.0,0.0000,64.0,270.32,1,0,5.094369
1,10.5,791.0,47.0,0.0000,134.0,197.85,1,0,3.169052
2,10.5,1087.0,80.0,0.0000,61.0,301.64,1,0,6.307375
3,10.5,598.0,271.0,15.6638,68.0,82.30,1,0,0.862217
5,10.5,3407.0,182.0,0.0000,112.0,1065.02,1,0,24.214638
...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,1,0,2.106037
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,1,0,10.871789
17173,10.5,263.0,9.0,0.0000,110.0,76.20,1,0,1.045847
17174,10.5,858.0,115.0,0.0000,317.0,133.31,1,0,2.879981


In [153]:
x.corr()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,pick_drop_date_diff,distance
additional_fare,1.000000,-0.000774,0.197606,0.199606,0.002352,0.315285,-0.041040,-0.001157,-0.000432
duration,-0.000774,1.000000,0.969788,0.968667,0.040577,0.865256,-0.113149,0.430620,0.012872
meter_waiting,0.197606,0.969788,1.000000,0.999474,0.030450,0.870126,-0.107379,0.412393,0.004354
meter_waiting_fare,0.199606,0.968667,0.999474,1.000000,0.029911,0.865073,-0.103418,0.410323,0.003912
meter_waiting_till_pickup,0.002352,0.040577,0.030450,0.029911,1.000000,0.045468,-0.035546,0.033834,0.003078
fare,0.315285,0.865256,0.870126,0.865073,0.045468,1.000000,-0.183861,0.402437,0.033631
label,-0.041040,-0.113149,-0.107379,-0.103418,-0.035546,-0.183861,1.000000,-0.092040,0.000714
pick_drop_date_diff,-0.001157,0.430620,0.412393,0.410323,0.033834,0.402437,-0.092040,1.000000,0.010453
distance,-0.000432,0.012872,0.004354,0.003912,0.003078,0.033631,0.000714,0.010453,1.000000


In [154]:
x = x.drop('label', axis=1)

In [155]:
x

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,pick_drop_date_diff,distance
0,10.5,834.0,56.0,0.0000,64.0,270.32,0,5.094369
1,10.5,791.0,47.0,0.0000,134.0,197.85,0,3.169052
2,10.5,1087.0,80.0,0.0000,61.0,301.64,0,6.307375
3,10.5,598.0,271.0,15.6638,68.0,82.30,0,0.862217
5,10.5,3407.0,182.0,0.0000,112.0,1065.02,0,24.214638
...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,0,2.106037
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,0,10.871789
17173,10.5,263.0,9.0,0.0000,110.0,76.20,0,1.045847
17174,10.5,858.0,115.0,0.0000,317.0,133.31,0,2.879981


In [156]:
x_test

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,pick_drop_date_diff,distance
0,10.5,924,42,2.44860,148,289.27,0,6.707807
1,10.5,4249,20,0.00000,91,1912.70,0,41.571559
2,10.5,1552,255,2.65880,23,394.00,0,5.918535
3,10.5,462,16,0.00000,198,154.32,0,3.302798
4,10.5,814,392,12.36920,69,147.47,0,2.589354
...,...,...,...,...,...,...,...,...
8571,10.5,1723,429,24.83332,3,388.48,0,3.935507
8572,10.5,1378,80,0.00000,125,379.85,0,7.519792
8573,10.5,418,56,3.28440,93,112.79,0,2.057870
8574,10.5,1604,548,31.67440,17,248.46,0,3.902113


In [157]:
x.describe()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,pick_drop_date_diff,distance
count,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000
mean,13.721437,1702.836987,628.875000,32.045122,112.489510,312.657884,0.006777,4.856351
std,98.992493,8121.556167,8064.657846,465.564240,442.129943,606.372103,0.082048,43.287935
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.500000,548.000000,33.000000,0.000000,26.000000,125.500000,0.000000,1.486629
50%,10.500000,1009.000000,123.000000,0.115600,73.000000,211.050000,0.000000,3.217561
75%,10.500000,1802.250000,351.000000,14.334400,144.000000,366.205000,0.000000,6.153272
max,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,26945.990000,1.000000,5611.282395


# ~~ Over sample training set ~~

In [158]:
# ros = RandomOverSampler()
# x, y = ros.fit_sample(x, y)

# rus = RandomUnderSampler()
# x_train, y_train = rus.fit_sample(x_train, y_train)

In [159]:
x.shape, y.shape, x_test.shape

((16968, 8), (16968,), (8576, 8))

# Models

In [160]:
models = {}

models['gnb'] = GaussianNB()
models['lr'] = LogisticRegression(solver='liblinear', multi_class='ovr',  max_iter=1000)
models['lda'] = LinearDiscriminantAnalysis()
models['knn'] = KNeighborsClassifier()
models['dt'] = DecisionTreeClassifier()

models['xgb'] = XGBClassifier() 
models['xgb5'] = XGBClassifier(base_score= 0.1, n_estimators=200, max_depth=5,  subsample = 0.7, colsample_bytree = 0.8, min_child_weight=3, eta = 0.3)

models['rf'] = RandomForestClassifier(random_state=RANDOM_SEED)
models['rf6'] = RandomForestClassifier(random_state=RANDOM_SEED, max_depth = 5 ,min_samples_leaf= 3, min_samples_split = 7, n_estimators = 300)

models['gb'] = GradientBoostingClassifier(random_state=RANDOM_SEED)
models['gb1'] = GradientBoostingClassifier(random_state=RANDOM_SEED, learning_rate = 0.05, max_depth = 15, n_estimators =500, subsample = 0.6)
models['gbo'] = GradientBoostingClassifier(random_state=RANDOM_SEED, learning_rate = 0.05, n_estimators =1000)


# stacked classifier
level0 = list()
# level0.append(('lr', LogisticRegression(penalty="l2", max_iter=1000)))
level0.append(('gbo',GradientBoostingClassifier(random_state=RANDOM_SEED, learning_rate = 0.05, n_estimators =1000)))
level0.append(('xgb1', XGBClassifier(n_estimators=200, max_depth=10)))
level0.append(('rf', RandomForestClassifier(random_state=RANDOM_SEED)))

level0.append(('lr', LogisticRegression(random_state=RANDOM_SEED,penalty="l2", max_iter=1000, C=2)))
level0.append(('svm', SVC(random_state=RANDOM_SEED,gamma=10)))
level0.append(('knn', KNeighborsClassifier(n_neighbors=10)))
level0.append(('sgd', SGDClassifier(random_state=RANDOM_SEED,loss="log", penalty="l2")))
level0.append(('ad', AdaBoostClassifier(random_state=RANDOM_SEED)))

# define meta learner model
level1 = LogisticRegression(penalty="l2", max_iter=1000, C=3, random_state=RANDOM_SEED)
# define the stacking ensemble
models['stacking'] =  StackingClassifier(estimators=level0, final_estimator=level1, cv=3)

# models['cat'] = CatBoostClassifier(iterations=2, learning_rate=1, depth=2)


# NN


In [161]:
# import tensorflow as tf
# inputs = tf.keras.Input(shape=(7,), name='Input')
# l0 = tf.keras.layers.Dense(256, activation='relu')
# a_0 = l1(inputs)
# l1 = tf.keras.layers.Dense(128, activation='relu')
# a = l1(a_0)
# l2 = tf.keras.layers.Dense(32, activation='relu')
# b = l2(a)
# l3 = tf.keras.layers.Dense(16, activation='relu')
# c = l3(b)
# output = tf.keras.layers.Dense(1, activation='softmax')
# outputs = output(c)
# model = tf.keras.Model(inputs, outputs)

In [162]:
# model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [163]:
# model.summary()

In [164]:
# X_train, X_eval, y_train, y_eval = train_test_split(x, y, test_size=0.33, shuffle=True, random_state=RANDOM_SEED)

In [165]:
# model.fit(X_train,y_train, epochs=10)

In [166]:
# y_pred =  model.predict(X_eval)
# f1 = f1_score(y_eval,y_pred)

In [167]:
# f1

# Evaluate using 3 fold cross validation

In [168]:
for i in models:
    model = models[i]
    skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
    model_results = cross_val_score(model, x, y, cv=skfold, scoring='f1')
    print(i, model_results, mean(model_results))

gnb [0.94740811 0.95270144 0.95178354] 0.9506310286497192
lr [0.95970009 0.95821258 0.95707353] 0.9583287371963273
lda [0.95399289 0.9558796  0.95324844] 0.9543736466797484
knn [0.96950057 0.96895471 0.96915461] 0.9692032977965036
dt [0.95593683 0.9580223  0.95559227] 0.9565171308340661
xgb [0.96912879 0.96885355 0.96881812] 0.9689334849260531
xgb5 [0.97502631 0.97305532 0.9711603 ] 0.9730806419679132
rf [0.97403589 0.97298326 0.97182157] 0.9729469057438846
rf6 [0.95721078 0.95841104 0.95824012] 0.9579539788385276
gb [0.97030642 0.97007009 0.96952435] 0.9699669545363422
gb1 [0.97522716 0.97423664 0.972528  ] 0.9739972679087268
gbo [0.97342256 0.97255575 0.97322624] 0.9730681821895408
stacking [0.97476734 0.97451074 0.97260406] 0.9739607126117845


# Cross Validate & Bag

In [ ]:
best_three_folds = []
for i in models:
    skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
    model = models[i]
    model_result = []


    for train_index, test_index in skfold.split(x, y):
        X_train, X_test = x.iloc[train_index,:], x.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train,y_train)
        y_pred = model.predict(X_test) #predict for cross-validate Xtest
        f1 = f1_score(y_test,y_pred)
        model_result.append(f1)

        #predict submission for gradient boost
        if i == 'gbo':
            prediction = model.predict(x_test)
            best_three_folds.append(prediction.tolist())
#             print(best_three_folds)
#             best_three_folds.append(prediction)

    print(i,model_result, mean(model_result))

# Predict by averaging fold results

In [ ]:
best_three_folds = np.array(best_three_folds)
best_three_folds = np.mean(best_three_folds, axis=0)
p = np.where(best_three_folds < 0.5, 0, 1)
p

# ~~ Train test split ~~

In [ ]:
# X_train, X_eval, y_train, y_eval = train_test_split(x, y, test_size=0.33, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# for i in models:
#   model = models[i]
#   model.fit(X_train,y_train)
#   y_pred = model.predict(X_eval)
#   f1 = f1_score(y_eval,y_pred)
#   print(i, f1)

gbo 0.9722624915434427


In [ ]:
# model =  models['xgbnew5']
# model.fit(x,y)
# p = model.predict(x_test)

# Write Submission

In [ ]:
out_df = pd.DataFrame()
out_df ['tripid'] = trip_id_no_norm
out_df['prediction'] = p
out_df
out_df.to_csv('submission.csv', index=False)